In [1]:
"""
This training script can be run both on a single gpu in debug mode,
and also in a larger training run with distributed data parallel (ddp).

To run on a single GPU, example:
$ python train.py --batch_size=32 --compile=False

To run with DDP on 4 gpus on 1 node, example:
$ torchrun --standalone --nproc_per_node=4 train.py

To run with DDP on 4 gpus across 2 nodes, example:
- Run on the first (master) node with example IP 123.456.123.456:
$ torchrun --nproc_per_node=8 --nnodes=2 --node_rank=0 --master_addr=123.456.123.456 --master_port=1234 train.py
- Run on the worker node:
$ torchrun --nproc_per_node=8 --nnodes=2 --node_rank=1 --master_addr=123.456.123.456 --master_port=1234 train.py
(If your cluster does not have Infiniband interconnect prepend NCCL_IB_DISABLE=1)
"""

import os
import time
import math
import pickle
import json
from contextlib import nullcontext

import pandas as pd

import numpy as np
import torch
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed import init_process_group, destroy_process_group

from model import GPTConfig, GPT

# -----------------------------------------------------------------------------
# default config values designed to train a gpt2 (124M) on OpenWebText
# I/O
out_dir = 'out-stock'
eval_interval = 100
log_interval = 1
eval_iters = 2
eval_only = False # if True, script exits right after the first eval
always_save_checkpoint = False # if True, always save a checkpoint after each eval
init_from = 'scratch' # 'scratch' or 'resume' or 'gpt2*'
# wandb logging
wandb_log = False # disabled by default
wandb_project = 'stock'
wandb_run_name = 'mini-gpt' # 'run' + str(time.time())
# data
dataset = 'stock'
gradient_accumulation_steps = 1 # used to simulate larger batch sizes
batch_size = 12 # if gradient_accumulation_steps > 1, this is the micro-batch size
block_size = 5
# model
n_layer = 8
n_head = 8
n_embd = 256 # 参数估计：params= layer*(12*emb**2), 数据量估计dataSize= 10 *params
# dropout = 0.998728434 # for pretraining 0 is good, for finetuning try 0.1+
dropout = 0
# dropout = 1-dataSize/10/params
bias = False # do we use bias inside LayerNorm and Linear layers?
meta_vocab_size = n_embd // 2

# adamw optimizer
learning_rate = 1e-3 # max learning rate
max_iters = 1000 # total number of training iterations
weight_decay = 1e-1
beta1 = 0.9
beta2 = 0.99
grad_clip = 1.0 # clip gradients at this value, or disable if == 0.0
# learning rate decay settings
decay_lr = True # whether to decay the learning rate
warmup_iters = 100 # how many steps to warm up for
lr_decay_iters = 2000 # should be ~= max_iters per Chinchilla
min_lr = 1e-4 # minimum learning rate, should be ~= learning_rate/10 per Chinchilla

# system
device = 'cpu' # examples: 'cpu', 'cuda', 'cuda:0', 'cuda:1' etc., or try 'mps' on macbooks
dtype = 'float16' # 'float32', 'bfloat16', or 'float16', the latter will auto implement a GradScaler
# -----------------------------------------------------------------------------
config_keys = [k for k,v in globals().items() if not k.startswith('_') and isinstance(v, (int, float, bool, str))]
config = {k: globals()[k] for k in config_keys} # will be useful for logging
# -----------------------------------------------------------------------------

master_process = True
seed_offset = 0


if master_process:
    os.makedirs(out_dir, exist_ok=True)
torch.manual_seed(1337 + seed_offset)
torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn
device_type = 'cuda' if 'cuda' in device else 'cpu' # for later use in torch.autocast
# note: float16 data type will automatically use a GradScaler
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)


In [59]:
np.random.seed(42)

datadir = os.path.join('data', dataset)

# meta数据
meta = {}
with open(os.path.join(datadir, 'meta.pkl'), 'r') as f:
    meta = json.load(f)
    meta_vocab_size = meta['vocab_size']
def decode(id):
    return meta['itos'][str(id)]
def encode(s):
    return [meta['stoi'][c] for c in s]

pd_train_data = pd.read_csv(os.path.join(datadir, 'train.csv')).iloc[1:,:]
pd_val_data = pd.read_csv(os.path.join(datadir, 'val.csv')).iloc[:,:]
pd_train_data



,trade_date,000001.SZ,000002.SZ,000004.SZ,000005.SZ,000006.SZ,000007.SZ,000008.SZ,000009.SZ,000010.SZ,...,688787.SH,688788.SH,688789.SH,688793.SH,688798.SH,688799.SH,688800.SH,688819.SH,688981.SH,689009.SH
1,20230104,1.0399,1.0461,1.0020,1.0000,1.0065,0.9740,1.0339,0.9943,1.0027,...,1.0326,1.0181,0.9764,1.0325,0.9924,0.9540,0.9688,0.9876,1.0091,0.9895
2,20230105,1.0112,1.0136,0.9890,1.0214,0.9610,0.9987,0.9959,1.0091,0.9786,...,0.9899,1.0018,1.0291,1.0578,1.0293,0.9946,1.0513,1.0180,0.9988,1.0106
3,20230106,1.0097,0.9943,0.9756,0.9895,0.9696,0.9949,0.9918,1.0016,1.0601,...,0.9796,0.9860,0.9648,0.9988,1.0071,0.9774,1.0200,1.0011,0.9955,0.9899
4,20230109,1.0123,0.9771,1.0083,0.9947,0.9930,1.0179,0.9959,0.9927,0.9588,...,1.0044,1.0039,1.0210,0.9675,1.0138,1.0074,0.9701,1.0071,0.9964,1.0205
5,20230110,0.9757,1.0011,0.9907,0.9894,0.9860,1.0013,0.9917,0.9951,0.9677,...,1.0113,0.9972,0.9745,0.9954,1.0463,0.9957,0.9852,1.0016,1.0007,0.9867
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262,20240130,0.9794,0.9732,0.9208,0.9802,0.9493,1.0043,0.9735,0.9625,0.9852,...,1.0338,0.9726,0.9428,1.0244,0.9815,0.9856,0.9759,0.9574,0.9499,0.9983
263,20240131,0.9958,0.9806,0.9001,0.9495,0.9287,0.9490,0.9682,0.9359,0.9248,...,0.8445,0.9485,0.9850,0.9745,0.9200,0.9520,0.9382,0.9984,1.0049,0.9661
264,20240201,0.9947,0.9833,0.9725,0.9468,0.9424,0.9597,0.9812,0.9769,0.9837,...,0.9960,0.9591,0.9995,1.0092,1.0473,0.9867,0.9661,0.9891,0.9859,1.0240
265,20240202,0.9851,1.0053,0.9459,0.9551,1.0051,0.9814,0.9809,0.9896,0.9421,...,0.9210,0.9438,0.9696,0.9926,0.9247,0.9500,0.9446,0.9824,0.9871,1.0033


In [63]:
decode(1)
# test = pd_train_data.iloc[:,1:]
# test.loc[test > 1.22] = -50

# df = test
# df = df.mask(df > 1.22, -50)
# (df > 1.2).sum().sum()

'000002.SZ'

In [67]:
# 写个python程序，从pandas数据中获取一批训练数据：
# 1. 读取pandas数据A，格式是：[trade_date, label1, label2, label3，。。。]，date的样例有：20230104，label*是数字，样例有1.0399、0.9943
# 2. X是生成指定格式的shape = [shape, ]




import random

random.seed(42)
torch.manual_seed(33)

print(pd_train_data.shape)

def print_data_info(split, index_data, index_id):
    data = pd_train_data if split == 'train' else pd_val_data
    data = pd_train_data

    print(f'{split}, code is {decode(index_id)}, date is {data.iloc[index_data, 0]},price_chg is {data.iloc[index_data, (index_id+1)*2]}, aount_chg is {data.iloc[index_data, (index_id+1)*2-1]}')


def get_batch(split):
    data = pd_train_data if split == 'train' else pd_val_data
    data = pd_train_data

    indices = [random.randint(0, len(data) -1 - block_size) for _ in range(batch_size)]
    
    x = torch.stack([data[i:i+block_size] for i in indices])
    y_indices = [torch.arange(0, len(x[i][0]) // 2) for i, j in enumerate(indices)]
    
    y_list = []
    for i, j in enumerate(indices):
        next_row = data[j + 1:j + 1 + block_size]
        price_chg_cols = [2 * col + 1 for col in y_indices[i]]
        
        topk = 3
        topk_values, topk_indices = next_row[:, price_chg_cols].topk(topk, dim=1)

        random_indices = torch.randint(0, topk, (topk_indices.size(0),))
        selected_indices = topk_indices[torch.arange(topk_indices.size(0)), random_indices]

        y_list.append(selected_indices.clone().detach())
        if i ==0 :
            print_data_info(split, (j+1).item(), selected_indices[0].item())
        
        

    y = torch.stack(y_list)

    x, y = x.to(device), y.to(device)
    return x, y

pd_train_data.iloc[-10:,]



10
(266, 5108)


,trade_date,000001.SZ,000002.SZ,000004.SZ,000005.SZ,000006.SZ,000007.SZ,000008.SZ,000009.SZ,000010.SZ,...,688787.SH,688788.SH,688789.SH,688793.SH,688798.SH,688799.SH,688800.SH,688819.SH,688981.SH,689009.SH
257,20240123,1.0066,1.0144,0.9956,1.0500,1.0120,0.9606,1.0142,1.0226,0.9961,...,1.0557,0.9838,1.0198,0.9571,1.1345,0.9906,1.0190,1.0121,0.9975,0.9770
258,20240124,1.0186,1.0251,1.0104,0.9905,1.0591,1.0410,1.0326,1.0274,1.0984,...,0.9685,1.0031,1.0215,0.9682,0.9865,0.9602,0.9839,1.0012,1.0145,1.0033
259,20240125,1.0182,1.0584,1.0684,1.0000,1.0379,1.0493,1.0270,1.0077,1.0072,...,1.0363,1.0922,1.0161,1.0171,1.0358,1.0603,1.0262,1.0359,1.0417,1.0325
260,20240126,1.0126,1.0241,1.0179,1.0192,1.0129,1.0493,1.0088,0.9940,1.0000,...,0.9861,0.9550,0.9681,0.9899,0.9308,0.9995,0.9723,0.9955,0.9839,0.9849
261,20240129,1.0083,0.9863,1.0081,0.9528,1.0042,1.0492,0.9826,1.0283,0.9609,...,1.0431,0.9804,0.9629,0.9629,0.9701,0.9810,0.9505,0.9667,0.9753,0.9790
262,20240130,0.9794,0.9732,0.9208,0.9802,0.9493,1.0043,0.9735,0.9625,0.9852,...,1.0338,0.9726,0.9428,1.0244,0.9815,0.9856,0.9759,0.9574,0.9499,0.9983
263,20240131,0.9958,0.9806,0.9001,0.9495,0.9287,0.9490,0.9682,0.9359,0.9248,...,0.8445,0.9485,0.9850,0.9745,0.9200,0.9520,0.9382,0.9984,1.0049,0.9661
264,20240201,0.9947,0.9833,0.9725,0.9468,0.9424,0.9597,0.9812,0.9769,0.9837,...,0.9960,0.9591,0.9995,1.0092,1.0473,0.9867,0.9661,0.9891,0.9859,1.0240
265,20240202,0.9851,1.0053,0.9459,0.9551,1.0051,0.9814,0.9809,0.9896,0.9421,...,0.9210,0.9438,0.9696,0.9926,0.9247,0.9500,0.9446,0.9824,0.9871,1.0033
266,20240205,1.0119,0.9673,0.8996,0.9529,0.9038,0.9501,0.9463,0.9579,0.8991,...,0.8250,0.9400,0.9620,0.8834,0.9171,0.8772,0.8671,0.9554,1.0199,1.0129


In [263]:
ix = pd.DataFrame(range(2, n_embd+1, 2))


filter = pd_val_data['trade_date']==20221205
(pd_val_data[filter]).iloc[0,range(2, n_embd+1, 2)][:15]
# pd_val_data[filter]
# pd_train_data.loc[,ix]
# pd_train_data['trade_date']=='000014.SZ'

000001.SZ_close_chg    1.0488
000002.SZ_close_chg    1.0288
000004.SZ_close_chg    1.0497
000005.SZ_close_chg    1.0476
000006.SZ_close_chg    0.9247
000007.SZ_close_chg    1.0079
000008.SZ_close_chg    1.0492
000009.SZ_close_chg    1.0039
000010.SZ_close_chg    1.0496
000011.SZ_close_chg    1.0210
000012.SZ_close_chg    1.0152
000014.SZ_close_chg    1.0999
000016.SZ_close_chg    1.0201
000017.SZ_close_chg    1.0154
000019.SZ_close_chg    1.0180
Name: 44, dtype: float64

In [264]:
# init these up here, can override if init_from='resume' (i.e. from a checkpoint)
iter_num = 0
best_val_loss = 1e9

# model init
model_args = dict(n_layer=n_layer, n_head=n_head, n_embd=n_embd, block_size=block_size,
                  bias=bias, vocab_size=meta_vocab_size, dropout=dropout) # start with model_args from command line
if init_from == 'scratch':
    # init a new model from scratch
    print("Initializing a new model from scratch")
    gptconf = GPTConfig(**model_args)
    model = GPT(gptconf)


model.to(device)

# initialize a GradScaler. If enabled=False scaler is a no-op
scaler = torch.cuda.amp.GradScaler(enabled=(dtype == 'float16'))

# optimizer
optimizer = model.configure_optimizers(weight_decay, learning_rate, (beta1, beta2), device_type)

checkpoint = None # free up memory



# helps estimate an arbitrarily accurate loss over either split using many batches
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            with ctx:
                logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# learning rate decay scheduler (cosine with warmup)
def get_lr(it):
    # 1) linear warmup for warmup_iters steps
    if it < warmup_iters:
        return learning_rate * it / warmup_iters
    # 2) if it > lr_decay_iters, return min learning rate
    if it > lr_decay_iters:
        return min_lr
    # 3) in between, use cosine decay down to min learning rate
    decay_ratio = (it - warmup_iters) / (lr_decay_iters - warmup_iters)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio)) # coeff ranges 0..1
    return min_lr + coeff * (learning_rate - min_lr)

# logging
if wandb_log and master_process:
    import wandb
    wandb.init(project=wandb_project, name=wandb_run_name, config=config)

X, Y = get_batch('train')

# 遍历并打印模型中的所有权重参数
# for name, param in model.named_parameters():
#     print(f"{name}: {param.shape}, {param}")

print(f'X is {X}')
print(f'Y is {Y}')
logits, loss = model(X, Y)
print(logits)
# print(f'logits is {logits}, shape is {logits.shape}')
# print(f'loss is {loss}, loss.shape is {loss.shape}')


# print(model)


Initializing a new model from scratch
number of parameters: 0.82M
num decayed parameter tensors: 6, with 827,392 parameters
num non-decayed parameter tensors: 3, with 768 parameters
using fused AdamW: False
train, code is 000514.SZ, date is 20220428,price_chg is 1.1006, aount_chg is 1.4199
X is tensor([[[1.5762, 1.0625, 1.6641,  ..., 1.0547, 1.3584, 1.0742],
         [0.6211, 1.0000, 0.8750,  ..., 0.9741, 0.9321, 0.9785],
         [1.7549, 0.9790, 1.2441,  ..., 1.0410, 0.8906, 1.0430],
         ...,
         [1.1299, 0.9712, 0.8252,  ..., 1.0020, 2.0215, 1.1006],
         [0.7407, 1.0146, 1.0430,  ..., 1.0166, 0.8755, 0.9497],
         [3.1270, 1.0439, 2.3027,  ..., 0.9736, 0.7158, 0.9595]],

        [[0.6177, 1.0078, 0.9966,  ..., 1.0000, 1.1885, 1.0391],
         [2.7031, 1.0352, 0.9189,  ..., 0.9849, 1.8320, 1.0391],
         [0.4780, 0.9834, 0.6621,  ..., 1.0000, 2.0117, 1.0889],
         ...,
         [1.0957, 0.9995, 0.9268,  ..., 0.9810, 0.8286, 0.9971],
         [0.6025, 1.0020

/home/ren/miniconda3/envs/nanogpt/lib/python3.12/site-packages/torch/cuda/amp/grad_scaler.py:126: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


In [265]:


# training loop
# X, Y = get_batch('train') # fetch the very first batch
t0 = time.time()
local_iter_num = 0 # number of iterations in the lifetime of this process
raw_model = model # unwrap DDP container if needed
running_mfu = -1.0
while True:
    # evaluate the loss on train/val sets and write checkpoints
    if iter_num % eval_interval == 0 and master_process:
        losses = estimate_loss()
        print(f"step {iter_num}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        
        if losses['val'] < best_val_loss or always_save_checkpoint:
            best_val_loss = losses['val']
            if iter_num > 0:
                checkpoint = {
                    'model': raw_model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'model_args': model_args,
                    'iter_num': iter_num,
                    'best_val_loss': best_val_loss,
                    'config': config,
                }
                print(f"saving checkpoint to {out_dir}")
                torch.save(checkpoint, os.path.join(out_dir, 'ckpt.pt'))
                
    if iter_num == 0 and eval_only:
        break
    
    X, Y = get_batch('train')
    logits, loss = model(X, Y)

    optimizer.zero_grad()
    loss.backward()

    optimizer.step()

    # timing and logging
    t1 = time.time()
    dt = t1 - t0
    t0 = t1
    if iter_num % log_interval == 0 and master_process:
        # get loss as float. note: this is a CPU-GPU sync point
        # scale up to undo the division above, approximating the true total loss (exact would have been a sum)
        lossf = loss.item() * gradient_accumulation_steps
        if local_iter_num >= 5: # let the training loop settle a bit
            mfu = raw_model.estimate_mfu(batch_size * gradient_accumulation_steps, dt)
            running_mfu = mfu if running_mfu == -1.0 else 0.9*running_mfu + 0.1*mfu
        print(f"iter {iter_num}: loss {lossf:.4f}, time {dt*1000:.2f}ms, mfu {running_mfu*100:.2f}%")
    iter_num += 1
    local_iter_num += 1

    # termination conditions
    if iter_num > max_iters:
        break



train, code is 000420.SZ, date is 20210823,price_chg is 1.099, aount_chg is 2.3759
train, code is 000158.SZ, date is 20181130,price_chg is 1.0354, aount_chg is 1.3248
val, code is 000541.SZ, date is 20200522,price_chg is 1.1, aount_chg is 1.1607
val, code is 000100.SZ, date is 20211209,price_chg is 1.052, aount_chg is 4.6886
step 0: train loss 4.9025, val loss 4.9060
train, code is 000423.SZ, date is 20211019,price_chg is 1.0672, aount_chg is 0.906
iter 0: loss 4.8730, time 85.20ms, mfu -100.00%
train, code is 000017.SZ, date is 20210708,price_chg is 1.0997, aount_chg is 2.3198
iter 1: loss 4.8472, time 22.05ms, mfu -100.00%
train, code is 000035.SZ, date is 20191230,price_chg is 1.0501, aount_chg is 2.7582
iter 2: loss 4.8366, time 22.83ms, mfu -100.00%
train, code is 000408.SZ, date is 20191024,price_chg is 1.0577, aount_chg is 4.2097
iter 3: loss 4.7616, time 23.26ms, mfu -100.00%
train, code is 000488.SZ, date is 20210108,price_chg is 1.0591, aount_chg is 1.0871
iter 4: loss 4.7583